## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

# Import API key
from dotenv import load_dotenv
load_dotenv()
g_key = os.getenv("Google_API_KEY")

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_Database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Souillac,GA,-20.5167,57.5167,69.58,66,72,13.65,broken clouds
1,1,Hobart,GA,-42.8794,147.3294,41.25,91,20,1.14,few clouds
2,2,Namibe,GA,-15.1961,12.1522,62.67,89,23,1.68,few clouds
3,3,San Matias,GA,-16.3667,-58.4000,83.97,26,35,4.38,scattered clouds
4,4,Moorhead,GA,46.8739,-96.7676,85.98,40,0,10.36,clear sky
...,...,...,...,...,...,...,...,...,...,...
700,700,Geraldton,GA,-28.7667,114.6000,54.01,87,100,9.22,overcast clouds
701,701,Vicksburg,GA,32.3526,-90.8779,93.43,62,0,11.50,clear sky
702,702,Chardara,GA,41.2547,67.9692,82.20,23,0,10.31,clear sky
703,703,Kropotkin,GA,45.4375,40.5756,59.09,65,2,6.89,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
3,3,San Matias,GA,-16.3667,-58.4000,83.97,26,35,4.38,scattered clouds
4,4,Moorhead,GA,46.8739,-96.7676,85.98,40,0,10.36,clear sky
7,7,Kapaa,GA,22.0752,-159.3190,84.52,69,40,16.11,scattered clouds
16,16,Hilo,GA,19.7297,-155.0900,84.27,91,75,5.75,moderate rain
22,22,Dwarka,GA,22.2394,68.9678,82.80,81,100,18.79,overcast clouds
24,24,Anloga,GA,5.7947,0.8973,75.25,80,12,12.93,few clouds
31,31,Alta Floresta,GA,-9.8756,-56.0861,80.33,33,0,1.07,clear sky
41,41,Sabi,GA,13.2333,-14.2000,75.38,96,100,6.26,heavy intensity rain
45,45,Itarema,GA,-2.9248,-39.9167,78.96,78,45,14.34,scattered clouds
50,50,Atuona,GA,-9.8000,-139.0333,77.40,73,14,16.15,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.count()


City_ID                236
City                   236
Country                236
Lat                    236
Lng                    236
Max Temp               236
Humidity               236
Cloudiness             236
Wind Speed             236
Weather Description    236
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()



In [8]:
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
3,San Matias,GA,83.97,scattered clouds,-16.3667,-58.4000,
4,Moorhead,GA,85.98,clear sky,46.8739,-96.7676,
7,Kapaa,GA,84.52,scattered clouds,22.0752,-159.3190,
16,Hilo,GA,84.27,moderate rain,19.7297,-155.0900,
22,Dwarka,GA,82.80,overcast clouds,22.2394,68.9678,
24,Anloga,GA,75.25,few clouds,5.7947,0.8973,
31,Alta Floresta,GA,80.33,clear sky,-9.8756,-56.0861,
41,Sabi,GA,75.38,heavy intensity rain,13.2333,-14.2000,
45,Itarema,GA,78.96,scattered clouds,-2.9248,-39.9167,
50,Atuona,GA,77.40,few clouds,-9.8000,-139.0333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

   
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
3,San Matias,GA,83.97,scattered clouds,-16.3667,-58.4000,"HOTEL ""SAN JOSE"""
4,Moorhead,GA,85.98,clear sky,46.8739,-96.7676,"Courtyard by Marriott Fargo Moorhead, MN"
7,Kapaa,GA,84.52,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
16,Hilo,GA,84.27,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
22,Dwarka,GA,82.80,overcast clouds,22.2394,68.9678,Hotel Narayan Avezika Comfort
24,Anloga,GA,75.25,few clouds,5.7947,0.8973,Pin Drop Hotel
31,Alta Floresta,GA,80.33,clear sky,-9.8756,-56.0861,Floresta Amazonica Hotel
41,Sabi,GA,75.38,heavy intensity rain,13.2333,-14.2000,
45,Itarema,GA,78.96,scattered clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
50,Atuona,GA,77.40,few clouds,-9.8000,-139.0333,Villa Enata


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Weather Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))